# Function Demonstrations

In [64]:
# These files are the source code for the functions
include("./RG_functions.jl")

predict_neural_network

## Logistic Regression

Create data that is approximately linearly separable.

In [4]:
X = rand(500,2)
y = zeros(500)
pos_ind = []
neg_ind = []
for i=1:size(X,1)
    if X[i,2] > 1.5*X[i,1] - .25
        y[i] = 1
        push!(pos_ind,i)
    else
        y[i] = 0
        push!(neg_ind,i)
    end
end

#class_pos = X[pos_ind,:]
#class_neg = X[neg_ind,:]

#scatter(class_pos[:,1], class_pos[:,2], color="blue", label="Class 1")
#scatter!(class_neg[:,1], class_neg[:,2], color="red", label="Class 2")

#x = LinRange(0,1,100)
f(x) = 1.5*x - .25
#plot!(x, f.(x),xlims=(0,1),ylims=(0,1))

x = LinRange(1/5,4/5,10)
x_noise_class1 = f.(x) + 1/5 * (rand(10) .- 0.5)
x_noise_class2 = f.(x) + 1/5 * (rand(10) .- 0.5)
#scatter!(x, x_noise_class1, color="blue")
#scatter!(x, x_noise_class2, color="red")
Xx = vcat(X,hcat(x,x_noise_class1), hcat(x,x_noise_class2))
X = hcat(ones(size(Xx,1),1),Xx)   # append columns of 1's
y = vcat(y,ones(10),zeros(10));

Solve for parameters and create prediction function.

In [5]:
# Solve
n,p = size(X)
b = zeros(p)
b = solve_LR_coef(X,y,b)
# Report error
prob(x, β) = 1 / (1 + exp(-(dot(x,β) )))

# Prediction function
pred(x,b) = round(prob(x,b))

Number of iterations: 8.


pred (generic function with 1 method)

Report misclassification error.

In [6]:
misclass = 0
for i = 1:n
    if pred(X[i,:],b) == y[i]
        misclass += 1
    end
end
println("$misclass out of $n training observations misclassified.")

0 out of 520 training observations misclassified.


## Kernel Ridge

Generate data for regression.

In [8]:
X = transpose(BostonHousing.features())
y = transpose(BostonHousing.targets())
n,p = size(X)

(506, 13)

Compute parameters for two different basis functions and create prediction function.

In [9]:
# Radial Basis Function
s = 10
K_rbf(x,z) = exp(-(norm(x-z)^2) / (2*(s^2)))
# Polynomial Basis Function
c = 1; d = 2
K_poly(x,z) = (c + dot(x,z))^d

λ = 1
α_rbf = kernel_ridge(X,y,λ,K_rbf)
α_poly = kernel_ridge(X,y,λ,K_poly)

# Prediction function
pred_kernel_ridge(x,K,X,α) = sum( K(x,X[i,:]) * α[i] for i = 1:size(X,1))

pred_kernel_ridge (generic function with 1 method)

Report mean squared error for each basis function.

In [10]:
# Mean Squared Error
y_preds_rbf = zeros(n)
y_preds_poly = zeros(n)
for i = 1:n
    y_preds_rbf[i] = pred_kernel_ridge(X[i,:],K_rbf,X,α_rbf)
    y_preds_poly[i] = pred_kernel_ridge(X[i,:],K_poly,X,α_poly)
end
println("Mean square error with RBF kernel: $(Statistics.mean( (y_preds_rbf .- y).^2))")
println("Mean square error with polynomial kernel: $(Statistics.mean( (y_preds_poly .- y).^2))")

Mean square error with RBF kernel: 48.58030241248382
Mean square error with polynomial kernel: 6.363275739611623


## Proximal Gradient Descent

Generate data for regression and center it.

In [11]:
# Load data
X = transpose(BostonHousing.features())
y = transpose(BostonHousing.targets())
n,p = size(X)
# Center y and estimate β_0
y_centered = y .- Statistics.mean(y)
β_0 = Statistics.mean(y)
# Create matrix of centered X columns
X_centered = zeros(n,p)
for j in 1:p
    X_centered[:,j] = X[:,j] .- Statistics.mean(X[:,j])
end

Compute parameters and create prediction function.

In [12]:
# Initialize β
β_init = zeros(size(X_centered,2))
λ = 10000

β = prox_grad_desc(X_centered, y_centered, β_init, λ)
println("$β")

# Prediction function
pred(X,β,β_0) = β_0 .+ X * β

Number of iterations: 1000.
Max iterations reached.
Real[0; 0.0381765577924749; 0; 0; 0; 0; 0; 0; 0; -0.016327374889328842; 0; 0.010874369828524572; -0.1857394401823319]


pred (generic function with 2 methods)

Report mean squared error.

In [13]:
y_pred = pred(X,β,β_0)

MSE = Statistics.mean((y - y_pred).^2)
println("Mean squared error: $MSE")

Mean squared error: 74.18408330593503


## Elastic Net

Generate data for regression and center it.

In [14]:
# Load data
X = transpose(BostonHousing.features())
y = transpose(BostonHousing.targets())
n,p = size(X)
# Center y and estimate β_0
y_centered = y .- Statistics.mean(y)
β_0 = Statistics.mean(y)
# Create matrix of centered X columns
X_centered = zeros(n,p)
for j in 1:p
    X_centered[:,j] = X[:,j] .- Statistics.mean(X[:,j])
end

Compute parameters and create prediction function.

In [17]:
# Initialize β
β_init = zeros(size(X_centered,2))
λ = 10000
α = 0.1   # 1 means lasso, 0 means ridge

β = elastic_net(X_centered, y_centered, β_init, λ, α)
println("$β")

# Prediction function
pred(X,β,β_0) = β_0 .+ X * β

Number of iterations: 1000.
Max iterations reached.
Real[-0.041753603120333466; 0.05502475975765802; -0.019075219745038403; 0; 0; 0.015521427968780603; 0.0007620910771608188; -0.009899700036271004; 0.03733115888685481; -0.012845543597233947; -0.04954615426492346; 0.010518427530508552; -0.32525422243992397]


pred (generic function with 2 methods)

Report mean squared error.

In [18]:
y_pred = pred(X,β,β_0)

MSE = Statistics.mean((y - y_pred).^2)
println("Mean squared error: $MSE")

Mean squared error: 78.37089939829339


## (Linear) Support Vector Machines

Create data that is approximately linearly separable.

In [24]:
# Create linear boundary data
X = rand(500,2)
y = zeros(500)
pos_ind = []
neg_ind = []
for i=1:size(X,1)
    if X[i,2] > 1.5*X[i,1] - .25
        y[i] = 1
        push!(pos_ind,i)
    else
        y[i] = -1
        push!(neg_ind,i)
    end
end

#class_pos = X[pos_ind,:]
#class_neg = X[neg_ind,:]

#scatter(class_pos[:,1], class_pos[:,2], color="blue", label="Class 1")
#scatter!(class_neg[:,1], class_neg[:,2], color="red", label="Class 2")

#x = LinRange(0,1,100)
f(x) = 1.5*x - .25
#plot!(x, f.(x),xlims=(0,1),ylims=(0,1))

x = LinRange(1/5,4/5,10)
x_noise_class1 = f.(x) + 1/5 * (rand(10) .- 0.5)
x_noise_class2 = f.(x) + 1/5 * (rand(10) .- 0.5)
#scatter!(x, x_noise_class1, color="blue")
#scatter!(x, x_noise_class2, color="red")
Xx = vcat(X,hcat(x,x_noise_class1), hcat(x,x_noise_class2))
X = hcat(ones(size(Xx,1),1),Xx)   # append columns of 1's
y = vcat(y,ones(10),-ones(10));

Compute parameters and create prediction function.

In [26]:
# Set parameter
C = 100
# Solve
(β_0, β, ξ) = SVM(X,y,C)

# Prediction function
SVM_classifier(x,β_0,β) = sign(x'*β + β_0)

Academic license - for non-commercial use only - expires 2022-08-24
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 521 rows, 524 columns and 3120 nonzeros
Model fingerprint: 0x58800b31
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.00s
Presolved: 521 rows, 524 columns, 3120 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 4
 Free vars  : 4
 AA' NZ     : 2.600e+03
 Factor NZ  : 3.513e+03
 Factor Ops : 3.024e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.63345561e-10 -5.63345561e-1

SVM_classifier (generic function with 1 method)

Report misclassification error.

In [27]:
# Report error
misclass = 0
for i = 1:n
    if SVM_classifier(X[i,:], β_0, β) != y[i]
        misclass += 1
    end
end
println("$misclass out of $n training observations misclassified.")

11 out of 506 training observations misclassified.


## Kernel Support Vector Machines

Generate data that is approximately separable with a quadratic boundary.

In [28]:
# Quadratic Boundary Data
X = rand(500,2)
y = zeros(500)
pos_ind = []
neg_ind = []
for i=1:size(X,1)
    if X[i,2] > -(X[i,1]+.5)*(X[i,1]-1)
        y[i] = 1
        push!(pos_ind,i)
    else
        y[i] = -1
        push!(neg_ind,i)
    end
end

#class_pos = X[pos_ind,:]
#class_neg = X[neg_ind,:]

#scatter(class_pos[:,1], class_pos[:,2], color="blue", label="Class 1")
#scatter!(class_neg[:,1], class_neg[:,2], color="red", label="Class 2")

#x = LinRange(0,1,100)
f(x) = -(x+0.5)*(x-1)
#plot!(x, f.(x),xlims=(0,1),ylims=(0,1))

x = LinRange(0,1,10)
x_noise_class1 = f.(x) + 1/5 * (rand(10) .- 0.5)
x_noise_class2 = f.(x) + 1/5 * (rand(10) .- 0.5)
#scatter!(x, x_noise_class1, color="blue")
#scatter!(x, x_noise_class2, color="red")
X = vcat(X,hcat(x,x_noise_class1), hcat(x,x_noise_class2))
y = vcat(y,ones(10),-ones(10));

Compute parameters for two different basis functions and create prediction function.

In [29]:
# Radial Basis Function
s = 100
K_rbf(x,z) = exp(-(norm(x-z)^2) / (2*(s^2)))
# Polynomial Basis Function
c = 0; d = 2
K_poly(x,z) = (c + dot(x,z))^d

C = 10
α_rbf = kernel_SVM(X,y,C,K_rbf)
α_poly = kernel_SVM(X,y,C,K_poly)

# find index of max α component, let that be k 
k_rbf = argmax(α_rbf)
k_poly = argmax(α_poly)

b_rbf = y[k_rbf] - sum( α_rbf[i]*y[i]*K_rbf(X[k_rbf,:],X[i,:]) for i = 1:n)
b_poly = y[k_poly] - sum( α_poly[i]*y[i]*K_poly(X[k_poly,:],X[i,:]) for i = 1:n)

# Prediction function
kernel_SVM_classifier(x,K,X,α,b) = sign(sum( α[i]*y[i]*K(x,X[i,:]) for i = 1:size(X,1)) + b)

Academic license - for non-commercial use only - expires 2022-08-24
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 520 columns and 520 nonzeros
Model fingerprint: 0x53189ea5
Model has 135460 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [1e+00, 2e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolved: 1 rows, 520 columns, 520 nonzeros
Presolved model has 135460 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 6
 AA' NZ     : 2.100e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.08086002e+06  1.30260000e+06  6.00e+04 

kernel_SVM_classifier (generic function with 1 method)

Report misclassification error.

In [32]:
misclass_rbf = 0
misclass_poly = 0
for i = 1:n
    if kernel_SVM_classifier(X[i,:], K_rbf,X, α_rbf, b_rbf) != y[i]
        misclass_rbf += 1
    end
    if kernel_SVM_classifier(X[i,:], K_poly,X, α_poly, b_poly) != y[i]
        misclass_poly += 1
    end
end
println("RBF kernel: $misclass_rbf out of $n training observations misclassified.")
println("Polynomial kernel: $misclass_poly out of $n training observations misclassified.")

RBF kernel: 286 out of 506 training observations misclassified.
Polynomial kernel: 33 out of 506 training observations misclassified.


## Bagging Procedure

In [40]:
# Sample data
X = [ones(5)'; 2*ones(5)'; 3*ones(5)'; 4*ones(5)']
y = [1.0; 2.0; 3.0; 4.0]
# Number of boostrapp samples
B = 10

b_samples, ind_used, ind_not_used = bootstrapper(X,y,B)

# Observations and indices for the i-th bootstrapped sample
i = 1
X_b, y_b = b_samples[i]
println("X_$i = $X_b\ny_$i = $y_b")
println("Indices used: $(ind_used[i])")
println("Indices not used: $(ind_not_used[i])")

X_1 = [3.0 3.0 3.0 3.0 3.0; 4.0 4.0 4.0 4.0 4.0; 3.0 3.0 3.0 3.0 3.0; 2.0 2.0 2.0 2.0 2.0]
y_1 = [3.0, 4.0, 3.0, 2.0]
Indices used: Set(Any[4, 2, 3])
Indices not used: Set([1])


## Neural Networks

### NN Classification

Load MNIST handwritten digit data.

In [66]:
# Classication

# load partial training set
tr_size = 5000
train_x, train_y = MNIST.traindata(1:tr_size)
# load partial test set
te_size = 200
test_x,  test_y  = MNIST.testdata(1:te_size)

X = zeros(tr_size,784)
for i = 1:tr_size
    X[i,:] = reshape(train_x[:,:,i],1,784)
end
y = train_y[1:tr_size]

X_test = zeros(te_size,784)
for i = 1:te_size
    X_test[i,:] = reshape(test_x[:,:,i],1,784)
end
y_test = test_y;

Train neural network.

In [70]:
num_hidden_layers = 2
size_hidden_layers = 8
W,b = train_neural_network(X,y,num_hidden_layers,size_hidden_layers,activation="sigmoid",problem_type="classification",num_classes=10,num_epochs=1);

Sigmoid used.
Total loss is 21194.91070211043.
Average loss 4.2389821404220855 out of 5000 training images.
4499 of 5000 training images misclassified.


Update the weights.

In [71]:
W,b = update_neural_network(X,y,W,b,num_hidden_layers,size_hidden_layers,activation="sigmoid",problem_type="classification",num_classes=10,num_epochs=10);

Sigmoid used.
Total loss is 21194.91070211043.
Average loss 4.2389821404220855 out of 5000 training images.
4499 of 5000 training images misclassified.


Predict test data.

In [73]:
y_pred = predict_neural_network(X_test,W,b,num_hidden_layers,size_hidden_layers;activation="sigmoid",problem_type="classification",num_classes=10)

Sigmoid used.


200-element Vector{Float64}:
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 ⋮
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0

### NN Regression

In [74]:
# Regression
X_all = transpose(BostonHousing.features())
X = X_all[1:450,:]
X_test = X_all[451:end,:]
y_all = transpose(BostonHousing.targets())
y = y_all[1:450]
y_test = y_all[451:end];

In [75]:
num_hidden_layers = 2
size_hidden_layers = 10
W,b = train_neural_network(X,y,num_hidden_layers,size_hidden_layers,activation="sigmoid",problem_type="regression",num_epochs=5);

Sigmoid used.
Average loss 185.64078020187634 out of 450 training images.


In [76]:
W,b = update_neural_network(X,y,W,b,num_hidden_layers,size_hidden_layers,activation="sigmoid",problem_type="regression",num_epochs=5);

Sigmoid used.
Average loss 185.64064165738674 out of 450 training images.


Predict test data.

In [77]:
y_pred = predict_neural_network(X_test,W,b,num_hidden_layers,size_hidden_layers;activation="sigmoid",problem_type="regression")

Sigmoid used.


56-element Vector{Float64}:
 13.278975477497077
 13.279015575160738
 13.279015527074977
 13.279016382274545
 13.279016480147035
 13.279015840552347
 13.27901649174759
 13.279015408827094
 13.279017876734613
 13.27901752947357
 13.279016737502179
 13.27901580641361
 13.27901763328099
  ⋮
 13.278992190013174
 13.278993128541625
 13.278994509817132
 13.27899478303203
 13.278991849815988
 13.278990830545688
 13.278992219548739
 13.278977439258489
 13.278975685271032
 13.278974566841047
 13.278977337403289
 13.278977249551858